In [1]:
import pandas as pd

**Задание 1**

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [25]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
...,...,...,...,...
99999,671,6268,2.5,1065579370
100000,671,6269,4.0,1065149201
100001,671,6365,4.0,1070940363
100002,671,6385,2.5,1070979663


In [34]:
ratings_time = ratings.groupby('userId').agg([min, max]).reset_index()
ratings_time


userId movieId        rating        timestamp            
               min    max    min  max         min         max
0        1      31   3671    1.0  4.0  1260759108  1260759205
1        2      10    720    1.0  5.0   835355395   835356246
2        3      60  84236    2.0  5.0  1298861589  1298932787
3        4      10   4006    1.0  5.0   949778714   949982274
4        5       3  48385    1.5  5.0  1163373044  1163375145
..     ...     ...    ...    ...  ...         ...         ...
666    667       6   1199    2.0  5.0   847271220   847272234
667    668     296   6425    1.0  5.0   993613196   993613478
668    669     223   4015    2.0  5.0  1015829081  1015829766
669    670       1   2912    1.0  5.0   938781328   940944033
670    671       1   6565    1.0  5.0  1063500751  1074784735

[671 rows x 7 columns]

In [44]:
ratings_time['diff'] = ratings_time['timestamp']['max'] - ratings_time['timestamp']['min']

In [53]:
user_dif = ratings_time[['userId','diff']]
user_dif

,userId,diff
,,
0,1,97
1,2,851
2,3,71198
3,4,203560
4,5,2101
...,...,...
666,667,1014
667,668,282
668,669,685


In [18]:
user_rate = ratings.pivot_table(index = 'userId', columns = 'rating', values = 'timestamp', aggfunc = 'count', fill_value = 0, 
                    margins = True).reset_index()

In [19]:
user_rate

rating,userId,0.5,1.0,1.5,2.0,2.5,3.0,3.5,4.0,4.5,5.0,All
0,1,0,2,0,7,3,4,1,3,0,0,20
1,2,0,2,0,4,0,36,0,23,0,11,76
2,3,0,0,0,1,3,18,9,11,4,5,51
3,4,0,5,0,5,0,23,0,52,0,119,204
4,5,0,0,1,0,3,3,27,42,19,5,100
...,...,...,...,...,...,...,...,...,...,...,...,...
667,668,0,2,0,0,0,7,0,3,0,8,20
668,669,0,0,0,7,0,14,0,12,0,4,37
669,670,0,2,0,3,0,5,0,10,0,11,31
670,671,0,1,0,4,4,9,13,57,8,19,115


In [50]:
yser_count_rate = user_rate.loc[:, ['userId', 'All']]

In [54]:
yser_count_rate_more100 = yser_count_rate.loc[user_rate.All >= 100]
yser_count_rate_more100

rating,userId,All
3,4,204
4,5,100
7,8,116
14,15,1700
16,17,363
...,...,...
658,659,142
663,664,519
664,665,434
670,671,115


In [66]:
user_cons= user_dif.join(yser_count_rate_more100, how='inner')
user_cons

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\reshape\merge.py:618: UserWarning: merging between different levels can give an unintended result (2 levels on the left, 1 on the right)
  warnings.warn(msg, UserWarning)


,"(userId, )","(diff, )",userId,All
3,4,203560,4,204
4,5,2101,5,100
7,8,85187,8,116
14,15,471393496,15,1700
16,17,8053,17,363
...,...,...,...,...
655,656,3053,656,128
658,659,31609411,659,142
663,664,98180439,664,519
664,665,54131471,665,434


In [67]:
user_cons.columns = ['t1', 'diff', 'userid', 'all']

In [70]:
user_cons = user_cons[['userid', 'all', 'diff']].reset_index()
user_cons

,index,userid,all,diff
0,3,4,204,203560
1,4,5,100,2101
2,7,8,116,85187
3,14,15,1700,471393496
4,16,17,363,8053
...,...,...,...,...
258,655,656,128,3053
259,658,659,142,31609411
260,663,664,519,98180439
261,664,665,434,54131471


In [78]:
def diff_seс (diff):
    return diff /24 / 3600


In [79]:
user_cons['diff_seconds'] = user_cons['diff'].apply(diff_seс)
user_cons

,index,userid,all,diff,diff_seconds
0,3,4,204,203560,2.356019
1,4,5,100,2101,0.024317
2,7,8,116,85187,0.985961
3,14,15,1700,471393496,5455.943241
4,16,17,363,8053,0.093206
...,...,...,...,...,...
258,655,656,128,3053,0.035336
259,658,659,142,31609411,365.849664
260,663,664,519,98180439,1136.347674
261,664,665,434,54131471,626.521655


In [80]:
del user_cons['index']

In [81]:
user_cons

,userid,all,diff,diff_seconds
0,4,204,203560,2.356019
1,5,100,2101,0.024317
2,8,116,85187,0.985961
3,15,1700,471393496,5455.943241
4,17,363,8053,0.093206
...,...,...,...,...
258,656,128,3053,0.035336
259,659,142,31609411,365.849664
260,664,519,98180439,1136.347674
261,665,434,54131471,626.521655


In [82]:
user_cons.sort_values('diff_seconds', ascending=False)

,userid,all,diff,diff_seconds
140,380,1063,515789661,5969.787743
212,547,2391,501810535,5807.992303
161,427,294,476569256,5515.847870
3,15,1700,471393496,5455.943241
248,624,1735,457492507,5295.052164
...,...,...,...,...
43,110,120,1148,0.013287
80,217,104,981,0.011354
121,316,140,916,0.010602
244,609,140,848,0.009815


**Задание 2**

Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
аналогичную таблицу по типам выручки с указанием адреса клиента
Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

Дана статистика услуг перевозок клиентов компании по типам:

rzd - железнодорожные перевозки
auto - автомобильные перевозки
air - воздушные перевозки
client_base - адреса клиентов

In [83]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [84]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [85]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [86]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [94]:
client_base_all= client_base.merge(rzd, how='left', on='client_id')
client_base_all

,client_id,address,rzd_revenue
0,111,Комсомольская 4,1093.0
1,112,Энтузиастов 8а,2810.0
2,113,Левобережная 1а,10283.0
3,114,Мира 14,5774.0
4,115,ЗЖБИиДК 1,981.0
5,116,Строителей 18,NaN
6,117,Панфиловская 33,NaN
7,118,Мастеркова 4,NaN


In [95]:
client_base_all= client_base_all.merge(auto, how='left', on='client_id')
client_base_all

,client_id,address,rzd_revenue,auto_revenue
0,111,Комсомольская 4,1093.0,NaN
1,112,Энтузиастов 8а,2810.0,NaN
2,113,Левобережная 1а,10283.0,57483.0
3,114,Мира 14,5774.0,83.0
4,115,ЗЖБИиДК 1,981.0,912.0
5,116,Строителей 18,NaN,4834.0
6,117,Панфиловская 33,NaN,98.0
7,118,Мастеркова 4,NaN,NaN


In [96]:
client_base_all= client_base_all.merge(air, how='left', on='client_id')
client_base_all

,client_id,address,rzd_revenue,auto_revenue,air_revenue
0,111,Комсомольская 4,1093.0,NaN,NaN
1,112,Энтузиастов 8а,2810.0,NaN,NaN
2,113,Левобережная 1а,10283.0,57483.0,NaN
3,114,Мира 14,5774.0,83.0,NaN
4,115,ЗЖБИиДК 1,981.0,912.0,81.0
5,116,Строителей 18,NaN,4834.0,4.0
6,117,Панфиловская 33,NaN,98.0,13.0
7,118,Мастеркова 4,NaN,NaN,173.0


In [97]:
client_base_all = client_base_all.fillna(0)

In [98]:
client_base_all

,client_id,address,rzd_revenue,auto_revenue,air_revenue
0,111,Комсомольская 4,1093.0,0.0,0.0
1,112,Энтузиастов 8а,2810.0,0.0,0.0
2,113,Левобережная 1а,10283.0,57483.0,0.0
3,114,Мира 14,5774.0,83.0,0.0
4,115,ЗЖБИиДК 1,981.0,912.0,81.0
5,116,Строителей 18,0.0,4834.0,4.0
6,117,Панфиловская 33,0.0,98.0,13.0
7,118,Мастеркова 4,0.0,0.0,173.0


In [100]:
client_base_all_noadres = client_base_all[['client_id', 'rzd_revenue', 'auto_revenue','air_revenue']]
client_base_all_noadres

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,0.0,0.0
1,112,2810.0,0.0,0.0
2,113,10283.0,57483.0,0.0
3,114,5774.0,83.0,0.0
4,115,981.0,912.0,81.0
5,116,0.0,4834.0,4.0
6,117,0.0,98.0,13.0
7,118,0.0,0.0,173.0


**Задание 3**

В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:
У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?
Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?
Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.

1. Надо найти координаты магазинов по адресу.
2. Через диапазон значений найти близки координаты местоположения пользователя и обрезать лишние.
3. Если последовательность координат имеет превязаность по времени можно получить время посещения и протяженность этого посещения.

Или я неправильно пояла задачу?